In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./ohlcv_with_future.csv')

In [3]:
def calculateGainAndLoss(arr):
    loss = 0
    gain = 0
    
    for index, value in enumerate(arr):
        if(index == 0):
            continue
        diff = value - arr[index - 1]
        if(diff >= 0):
            gain += diff
        else:
            loss += -1 * diff
    
    return {
        "loss": loss,
        "gain": gain
    }
    


In [4]:
def calculateInitialRSI(n, df):
    close = df['close'].values
    initial = close[0:n]
    res = calculateGainAndLoss(initial)
    avg_loss = res['loss'] / 14
    avg_gain = res['gain'] / 14
    RS = avg_gain / avg_loss
    RSI = 100 - (100 / (1+RS))
    return {
        "avg_gain": avg_gain,
        "avg_loss": avg_loss,
        "RS": RS,
        "RSI": RSI
    }

In [20]:
def calculateRSI(n, df):
    close = df['close'].values
    tail = close[n:len(df)]
    temp = calculateInitialRSI(n, df)
    arr = []
    arr.append(temp["RSI"])
    for index, value in enumerate(tail):
        if(index == 0):
            continue
        else:
            diff = value - tail[index - 1]
            avg_gain = temp["avg_gain"]
            avg_loss = temp["avg_loss"]
            if(diff >= 0):
                avg_gain = (avg_gain * (n - 1) + diff) / n
                temp["avg_gain"] = avg_gain
            else:
                avg_loss = ((avg_loss * (n - 1)) + (-1 * diff)) / n
                temp["avg_loss"] = avg_loss
            RS = avg_gain / avg_loss
            RSI = 100 - (100 / (1+RS))
            temp["RS"] = RS
            temp ["RSI"] = RSI
            arr.append(RSI)   
    return arr  

In [33]:
arr = calculateRSI(14, df)
res = np.append(([None] * (len(df) - len(arr))), arr)
df['RSI'] = res
df.to_csv("./csv/ohlcv_with_future.csv")